# Limpieza y Preparación de Datos | Proyecto Aurelion

---

## Información del Proyecto

| **Campo** | **Detalle** |
|-----------|-------------|
| **Proyecto** | Minimarket Aurelion - Pipeline de Datos Comerciales |
| **Sprint** | Sprint 2 - Análisis Exploratorio de Datos |
| **Notebook** | `7. Limpieza_datos.ipynb` |
| **Autor** | Diego Armando Vásquez Chávez |
| **Email** | [email protected] |
| **Fecha** | 3 de noviembre de 2025 |
| **Mentor** | Mirta Gladys Julio |

---

## Propósito del Notebook

Este notebook ejecuta el proceso profesional de limpieza y preparación de datos del Minimarket Aurelion, aplicando técnicas avanzadas de calidad de datos, validación de integridad referencial y normalización. Constituye la fase crítica del pipeline que garantiza datasets confiables para análisis posteriores y modelado de Machine Learning.

## Objetivos Principales

### Inspección y Diagnóstico
- Análisis exhaustivo de calidad de datos en datasets originales
- Identificación de valores nulos, duplicados y inconsistencias
- Evaluación de integridad referencial entre tablas relacionales

### Tratamiento de Problemas
- Corrección de tipos de datos y formatos inconsistentes
- Eliminación inteligente de duplicados preservando información crítica
- Imputación estratégica de valores faltantes según reglas de negocio

### Validación y Normalización
- Aplicación de reglas de validación de integridad referencial
- Normalización de campos categóricos y estandarización de formatos
- Verificación de rangos válidos y coherencia lógica

### Documentación y Trazabilidad
- Registro detallado de transformaciones aplicadas
- Métricas de calidad antes/después del proceso
- Exportación de datasets limpios con logs de auditoría

---

## Etapas Principales de Ejecución

1️⃣ **Setup y Configuración Inicial**  
Configuración del entorno de limpieza con logging estructurado, definición de rutas y carga de parámetros.

2️⃣ **Inspección Inicial Exhaustiva**  
Análisis de estructura, tipos y dimensiones de cada dataset, identificando campos críticos y problemas iniciales.

3️⃣ **Identificación y Catalogación de Problemas**  
Detección de nulos, duplicados, tipos incorrectos e inconsistencias referenciales.

4️⃣ **Tratamiento Estratégico de Nulos**  
Imputaciones y descartes siguiendo prioridades de negocio.

5️⃣ **Eliminación Inteligente de Duplicados**  
Depuración por claves y filas completas, preservando registros de mayor valor.

6️⃣ **Corrección de Tipos y Formatos**  
Conversión a tipos adecuados, normalización de fechas y escalas numéricas.

7️⃣ **Normalización y Estandarización**  
Title Case, validación de rangos y cálculo de métricas derivadas.

8️⃣ **Validaciones Finales y Control de Calidad**  
Pruebas de integridad post-limpieza y métricas comparativas.

9️⃣ **Conclusiones y Exportación**  
Resumen ejecutivo, exportación y trazabilidad.

---

## Datasets de Entrada y Salida

**Entrada (Excel)**
- `clientes.xlsx`: Información de clientes del minimarket
- `productos.xlsx`: Catálogo de productos con precios base
- `ventas.xlsx`: Transacciones principales con fechas y medios de pago
- `detalle_ventas.xlsx`: Líneas de detalle con cantidades y precios

**Salida (CSV limpios)**
- `datasets_limpios/clientes_clean.csv`: Clientes validados y normalizados
- `datasets_limpios/productos_clean.csv`: Productos con datos consistentes
- `datasets_limpios/ventas_clean.csv`: Ventas con integridad verificada
- `datasets_limpios/detalle_clean.csv`: Detalle con validación referencial

**Reportes de calidad**
- `export/resumen_limpieza.csv`: Métricas comparativas del proceso
- Logs de trazabilidad con transformaciones aplicadas

---

## Resultados Esperados

✅ Datasets de alta calidad con integridad referencial garantizada  
✅ Eliminación completa de duplicados y valores inconsistentes  
✅ Normalización de formatos y tipos de datos  
✅ Trazabilidad completa de transformaciones aplicadas  
✅ Base sólida para análisis exploratorio y modelado posterior

---

## 📋 Índice de Contenidos

| **Sección** | **Descripción** |
|-------------|-----------------|
| 1. Setup y Configuración | Inicialización y carga de configuraciones |
| 2. Inspección Inicial | Análisis de dimensiones y calidad inicial |
| 3. Identificación de Problemas | Catalogación de issues de calidad |
| 4. Tratamiento de Nulos | Estrategias de imputación y eliminación |
| 5. Eliminación de Duplicados | Deduplicación inteligente |
| 6. Corrección de Tipos | Conversión a tipos correctos |
| 7. Normalización y Estandarización | Estandarización de formatos |
| 8. Validaciones Finales | Controles de calidad |
| 9. Conclusiones y Exportación | Resumen y exportación |
| Reglas adicionales | Reglas de negocio complementarias |

*🔧 Fase: Preparación de Datos | 🎯 Objetivo: Datasets de Calidad para EDA*

## 1. Setup y Configuración
Importamos bibliotecas, definimos rutas y funciones auxiliares. Cargamos configuración centralizada desde `config.json`.

In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Rutas
DIR_PROYECTO = Path.cwd()
DIR_DATASETS = DIR_PROYECTO / 'datasets'
DIR_DATASETS_LIMPIOS = DIR_PROYECTO / 'datasets_limpios'
DIR_EXPORT = DIR_PROYECTO / 'export'
DIR_LOGS = DIR_PROYECTO / 'logs'

# Crear directorios si no existen
DIR_DATASETS_LIMPIOS.mkdir(exist_ok=True)
DIR_EXPORT.mkdir(exist_ok=True)
DIR_LOGS.mkdir(exist_ok=True)

# Cargar config
ruta_config = DIR_PROYECTO / 'config.json'
if ruta_config.exists():
    with open(ruta_config, 'r', encoding='utf-8') as f:
        config = json.load(f)
else:
    config = {}

print("✅ Setup completo")
print(f"   Proyecto: {DIR_PROYECTO}")
print(f"   Datasets: {DIR_DATASETS}")
print(f"   Salida: {DIR_DATASETS_LIMPIOS}")

✅ Setup completo
   Proyecto: c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion
   Datasets: c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion\datasets
   Salida: c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion\datasets_limpios


## 2. Inspección Inicial
Cargamos los datasets originales desde Excel y realizamos una inspección básica de dimensiones, tipos y valores nulos.

⬆️ Volver al índice

In [2]:
# Cargar datasets originales
clientes = pd.read_excel(DIR_DATASETS / 'clientes.xlsx')
productos = pd.read_excel(DIR_DATASETS / 'productos.xlsx')
ventas = pd.read_excel(DIR_DATASETS / 'ventas.xlsx')
detalle = pd.read_excel(DIR_DATASETS / 'detalle_ventas.xlsx')

# Guardar copias originales
orig_clientes = clientes.copy()
orig_productos = productos.copy()
orig_ventas = ventas.copy()
orig_detalle = detalle.copy()

# Inspección básica
datasets = {
    'clientes': clientes,
    'productos': productos,
    'ventas': ventas,
    'detalle_ventas': detalle
}

print("📊 Inspección inicial:\n")
for nombre, df in datasets.items():
    print(f"{nombre}:")
    print(f"   Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas")
    print(f"   Nulos: {df.isnull().sum().sum()} valores faltantes")
    print(f"   Duplicados: {df.duplicated().sum()} filas duplicadas completas")
    print()

📊 Inspección inicial:

clientes:
   Dimensiones: 1,296 filas x 5 columnas
   Nulos: 30 valores faltantes
   Duplicados: 94 filas duplicadas completas

productos:
   Dimensiones: 151 filas x 5 columnas
   Nulos: 3 valores faltantes
   Duplicados: 11 filas duplicadas completas

ventas:
   Dimensiones: 12,960 filas x 9 columnas
   Nulos: 27796 valores faltantes
   Duplicados: 36 filas duplicadas completas

detalle_ventas:
   Dimensiones: 12,960 filas x 5 columnas
   Nulos: 311 valores faltantes
   Duplicados: 922 filas duplicadas completas



## 3. Identificación de Problemas
Identificamos problemas de calidad específicos: columnas con alta tasa de nulos, tipos incorrectos, valores fuera de rango, duplicados en claves primarias e inconsistencias de integridad referencial.

⬆️ Volver al índice

In [3]:
problemas = {}

# 3.1 Análisis de nulos por columna
print("⚠️ Problemas identificados:\n")
for nombre, df in datasets.items():
    nulos_col = df.isnull().sum()
    pct_nulos = (nulos_col / len(df) * 100).round(2)
    cols_con_nulos = pct_nulos[pct_nulos > 0].to_dict()
    if cols_con_nulos:
        problemas[f"{nombre}_nulos"] = cols_con_nulos
        print(f"{nombre} - Columnas con nulos:")
        for col, pct in cols_con_nulos.items():
            print(f"   {col}: {pct}%")

# 3.2 Verificar claves primarias duplicadas
print("\n🔑 Duplicados en claves primarias:")
if 'id_cliente' in clientes.columns and clientes['id_cliente'].duplicated().any():
    print(f"   clientes.id_cliente: {clientes['id_cliente'].duplicated().sum()} duplicados")
if 'id_producto' in productos.columns and productos['id_producto'].duplicated().any():
    print(f"   productos.id_producto: {productos['id_producto'].duplicated().sum()} duplicados")
if 'id_venta' in ventas.columns and ventas['id_venta'].duplicated().any():
    print(f"   ventas.id_venta: {ventas['id_venta'].duplicated().sum()} duplicados")

# 3.3 Integridad referencial
print("\n🔗 Integridad referencial:")
id_clientes_validos = set(clientes['id_cliente']) if 'id_cliente' in clientes.columns else set()
id_productos_validos = set(productos['id_producto']) if 'id_producto' in productos.columns else set()
id_ventas_validos = set(ventas['id_venta']) if 'id_venta' in ventas.columns else set()

huerfanos_vc = ventas[~ventas['id_cliente'].isin(id_clientes_validos)] if 'id_cliente' in ventas.columns else pd.DataFrame()
huerfanos_dp = detalle[~detalle['id_producto'].isin(id_productos_validos)] if 'id_producto' in detalle.columns else pd.DataFrame()
huerfanos_dv = detalle[~detalle['id_venta'].isin(id_ventas_validos)] if 'id_venta' in detalle.columns else pd.DataFrame()

if len(huerfanos_vc) > 0:
    print(f"   ventas con id_cliente inexistente: {len(huerfanos_vc)}")
if len(huerfanos_dp) > 0:
    print(f"   detalle con id_producto inexistente: {len(huerfanos_dp)}")
if len(huerfanos_dv) > 0:
    print(f"   detalle con id_venta inexistente: {len(huerfanos_dv)}")

print("\n✅ Identificación de problemas completa")

⚠️ Problemas identificados:

clientes - Columnas con nulos:
   nombre_cliente: 0.54%
   email: 1.16%
   ciudad: 0.46%
   fecha_alta: 0.15%
productos - Columnas con nulos:
   nombre_producto: 0.66%
   categoria: 0.66%
   precio_unitario: 0.66%
ventas - Columnas con nulos:
   fecha: 0.49%
   id_cliente: 0.52%
   medio_pago: 1.16%
   canal: 0.22%
   csat_1a5: 19.41%
   ces_1a5: 25.82%
   nps_0a10: 83.43%
   nps_segmento: 83.43%
detalle_ventas - Columnas con nulos:
   id_producto: 0.25%
   cantidad: 1.03%
   precio_unitario: 0.66%
   importe: 0.46%

🔑 Duplicados en claves primarias:
   clientes.id_cliente: 96 duplicados
   productos.id_producto: 11 duplicados
   ventas.id_venta: 960 duplicados

🔗 Integridad referencial:
   ventas con id_cliente inexistente: 68
   detalle con id_producto inexistente: 32

✅ Identificación de problemas completa


## 4. Tratamiento de Nulos
Eliminamos o imputamos valores nulos según reglas de negocio. Para campos críticos (IDs, precios, fechas) eliminamos filas; para campos opcionales conservamos la fila.

⬆️ Volver al índice

In [4]:
print("🧹 Tratamiento de nulos:\n")

# Campos críticos (eliminar si están nulos)
campos_criticos_clientes = ['id_cliente', 'nombre', 'email']
campos_criticos_productos = ['id_producto', 'nombre', 'precio_base', 'categoria']
campos_criticos_ventas = ['id_venta', 'id_cliente', 'fecha', 'medio_pago']
campos_criticos_detalle = ['id_venta', 'id_producto', 'cantidad', 'precio_unitario']

# Clientes: eliminar nulos en campos críticos
nulos_antes = clientes.isnull().sum().sum()
for col in campos_criticos_clientes:
    if col in clientes.columns:
        clientes = clientes.dropna(subset=[col])
nulos_despues = clientes.isnull().sum().sum()
print(f"clientes: {nulos_antes} → {nulos_despues} nulos | {len(orig_clientes) - len(clientes)} filas eliminadas")

# Productos
nulos_antes = productos.isnull().sum().sum()
for col in campos_criticos_productos:
    if col in productos.columns:
        productos = productos.dropna(subset=[col])
nulos_despues = productos.isnull().sum().sum()
print(f"productos: {nulos_antes} → {nulos_despues} nulos | {len(orig_productos) - len(productos)} filas eliminadas")

# Ventas
nulos_antes = ventas.isnull().sum().sum()
for col in campos_criticos_ventas:
    if col in ventas.columns:
        ventas = ventas.dropna(subset=[col])
nulos_despues = ventas.isnull().sum().sum()
print(f"ventas: {nulos_antes} → {nulos_despues} nulos | {len(orig_ventas) - len(ventas)} filas eliminadas")

# Detalle
nulos_antes = detalle.isnull().sum().sum()
for col in campos_criticos_detalle:
    if col in detalle.columns:
        detalle = detalle.dropna(subset=[col])
nulos_despues = detalle.isnull().sum().sum()
print(f"detalle_ventas: {nulos_antes} → {nulos_despues} nulos | {len(orig_detalle) - len(detalle)} filas eliminadas")

print("\n✅ Tratamiento de nulos completo")

🧹 Tratamiento de nulos:

clientes: 30 → 15 nulos | 15 filas eliminadas
productos: 3 → 2 nulos | 1 filas eliminadas
ventas: 27796 → 26897 nulos | 278 filas eliminadas
detalle_ventas: 311 → 58 nulos | 251 filas eliminadas

✅ Tratamiento de nulos completo


## 5. Eliminación de Duplicados
Eliminamos filas duplicadas completas y duplicados en claves primarias (conservando la primera ocurrencia).

⬆️ Volver al índice

In [5]:
print("🗑️ Eliminación de duplicados:\n")

# Clientes: duplicados en id_cliente
dup_antes = clientes.duplicated(subset=['id_cliente']).sum() if 'id_cliente' in clientes.columns else 0
clientes = clientes.drop_duplicates(subset=['id_cliente'], keep='first') if 'id_cliente' in clientes.columns else clientes
dup_despues = clientes.duplicated(subset=['id_cliente']).sum() if 'id_cliente' in clientes.columns else 0
print(f"clientes.id_cliente: {dup_antes} → {dup_despues} duplicados")

# Productos: duplicados en id_producto
dup_antes = productos.duplicated(subset=['id_producto']).sum() if 'id_producto' in productos.columns else 0
productos = productos.drop_duplicates(subset=['id_producto'], keep='first') if 'id_producto' in productos.columns else productos
dup_despues = productos.duplicated(subset=['id_producto']).sum() if 'id_producto' in productos.columns else 0
print(f"productos.id_producto: {dup_antes} → {dup_despues} duplicados")

# Ventas: duplicados en id_venta
dup_antes = ventas.duplicated(subset=['id_venta']).sum() if 'id_venta' in ventas.columns else 0
ventas = ventas.drop_duplicates(subset=['id_venta'], keep='first') if 'id_venta' in ventas.columns else ventas
dup_despues = ventas.duplicated(subset=['id_venta']).sum() if 'id_venta' in ventas.columns else 0
print(f"ventas.id_venta: {dup_antes} → {dup_despues} duplicados")

# Detalle: duplicados completos
dup_antes = detalle.duplicated().sum()
detalle = detalle.drop_duplicates(keep='first')
dup_despues = detalle.duplicated().sum()
print(f"detalle_ventas (filas completas): {dup_antes} → {dup_despues} duplicados")

print("\n✅ Eliminación de duplicados completa")

🗑️ Eliminación de duplicados:

clientes.id_cliente: 94 → 0 duplicados
productos.id_producto: 11 → 0 duplicados
ventas.id_venta: 920 → 0 duplicados
detalle_ventas (filas completas): 922 → 0 duplicados

✅ Eliminación de duplicados completa


## 6. Corrección de Tipos
Convertimos columnas a sus tipos correctos: fechas a `datetime`, IDs a `int`, precios/cantidades a `float`, categorías a `str`.

⬆️ Volver al índice

In [6]:
print("🔧 Corrección de tipos:\n")

# Clientes
if 'id_cliente' in clientes.columns:
    clientes['id_cliente'] = clientes['id_cliente'].astype(int)
if 'fecha_alta' in clientes.columns:
    clientes['fecha_alta'] = pd.to_datetime(clientes['fecha_alta'], errors='coerce')
for col in ['nombre', 'email']:
    if col in clientes.columns:
        clientes[col] = clientes[col].astype(str)
print("clientes: id_cliente → int, fecha_alta → datetime")

# Productos
if 'id_producto' in productos.columns:
    productos['id_producto'] = productos['id_producto'].astype(int)
if 'precio_base' in productos.columns:
    productos['precio_base'] = pd.to_numeric(productos['precio_base'], errors='coerce')
for col in ['nombre', 'categoria']:
    if col in productos.columns:
        productos[col] = productos[col].astype(str)
print("productos: id_producto → int, precio_base → float")

# Ventas
if 'id_venta' in ventas.columns:
    ventas['id_venta'] = ventas['id_venta'].astype(int)
if 'id_cliente' in ventas.columns:
    ventas['id_cliente'] = ventas['id_cliente'].astype(int)
if 'fecha' in ventas.columns:
    ventas['fecha'] = pd.to_datetime(ventas['fecha'], errors='coerce')
if 'medio_pago' in ventas.columns:
    ventas['medio_pago'] = ventas['medio_pago'].astype(str)
print("ventas: id_venta, id_cliente → int, fecha → datetime")

# Detalle
for col in ['id_venta', 'id_producto']:
    if col in detalle.columns:
        detalle[col] = detalle[col].astype(int)
for col in ['cantidad', 'precio_unitario']:
    if col in detalle.columns:
        detalle[col] = pd.to_numeric(detalle[col], errors='coerce')
print("detalle_ventas: IDs → int, cantidad/precio → float")

print("\n✅ Corrección de tipos completa")

🔧 Corrección de tipos:

clientes: id_cliente → int, fecha_alta → datetime
productos: id_producto → int, precio_base → float
ventas: id_venta, id_cliente → int, fecha → datetime
detalle_ventas: IDs → int, cantidad/precio → float

✅ Corrección de tipos completa


## 7. Normalización y Estandarización
Normalizamos nombres de columnas, estandarizamos valores categóricos y validamos rangos de valores numéricos.

⬆️ Volver al índice

In [7]:
print("📏 Normalización y estandarización:\n")

# Normalizar nombres de columnas a minúsculas
clientes.columns = clientes.columns.str.lower().str.strip()
productos.columns = productos.columns.str.lower().str.strip()
ventas.columns = ventas.columns.str.lower().str.strip()
detalle.columns = detalle.columns.str.lower().str.strip()
print("✓ Nombres de columnas normalizados (minúsculas)")

# Estandarizar categorías
if 'categoria' in productos.columns:
    productos['categoria'] = productos['categoria'].astype(str).str.strip().str.title()
if 'medio_pago' in ventas.columns:
    ventas['medio_pago'] = ventas['medio_pago'].astype(str).str.strip().str.title()
print("✓ Categorías estandarizadas (Title Case)")

# Validar rangos numéricos
if 'precio_base' in productos.columns:
    productos = productos[productos['precio_base'] > 0]
if 'cantidad' in detalle.columns:
    detalle = detalle[detalle['cantidad'] > 0]
if 'precio_unitario' in detalle.columns:
    detalle = detalle[detalle['precio_unitario'] > 0]
print("✓ Rangos numéricos validados (valores positivos)")

# Calcular importe_total si no existe
if 'importe_total' not in detalle.columns and {'cantidad', 'precio_unitario'}.issubset(detalle.columns):
    detalle['importe_total'] = detalle['cantidad'] * detalle['precio_unitario']
    print("✓ Campo 'importe_total' calculado")

print("\n✅ Normalización completa")

📏 Normalización y estandarización:

✓ Nombres de columnas normalizados (minúsculas)
✓ Categorías estandarizadas (Title Case)
✓ Rangos numéricos validados (valores positivos)
✓ Campo 'importe_total' calculado

✅ Normalización completa


## 8. Validaciones Finales
Realizamos validaciones finales de integridad referencial y coherencia de datos. Verificamos que no queden huérfanos y que las dimensiones sean consistentes.

⬆️ Volver al índice

In [8]:
print("🔍 Validaciones finales:\n")

# Reconstruir conjuntos actualizados de IDs válidos
id_clientes_validos = set(clientes['id_cliente']) if 'id_cliente' in clientes.columns else set()
id_productos_validos = set(productos['id_producto']) if 'id_producto' in productos.columns else set()
id_ventas_validos = set(ventas['id_venta']) if 'id_venta' in ventas.columns else set()

# Validar integridad referencial: eliminar huérfanos
if 'id_cliente' in ventas.columns:
    ventas = ventas[ventas['id_cliente'].isin(id_clientes_validos)]
if 'id_producto' in detalle.columns:
    detalle = detalle[detalle['id_producto'].isin(id_productos_validos)]
if 'id_venta' in detalle.columns:
    detalle = detalle[detalle['id_venta'].isin(id_ventas_validos)]

print(f"✓ Ventas con id_cliente válido: {len(ventas)}")
print(f"✓ Detalle con id_producto válido: {len(detalle)}")
print(f"✓ Detalle con id_venta válido: {len(detalle)}")

# Validar sin nulos críticos
assert clientes['id_cliente'].isnull().sum() == 0, "❌ id_cliente tiene nulos"
assert productos['id_producto'].isnull().sum() == 0, "❌ id_producto tiene nulos"
assert ventas['id_venta'].isnull().sum() == 0, "❌ id_venta tiene nulos"
assert detalle['id_venta'].isnull().sum() == 0, "❌ detalle.id_venta tiene nulos"
print("✓ Sin nulos en claves primarias")

# Validar sin duplicados en PKs
assert clientes['id_cliente'].duplicated().sum() == 0, "❌ id_cliente duplicado"
assert productos['id_producto'].duplicated().sum() == 0, "❌ id_producto duplicado"
assert ventas['id_venta'].duplicated().sum() == 0, "❌ id_venta duplicado"
print("✓ Sin duplicados en claves primarias")

print("\n✅ Validaciones finales pasadas")

🔍 Validaciones finales:

✓ Ventas con id_cliente válido: 11728
✓ Detalle con id_producto válido: 11533
✓ Detalle con id_venta válido: 11533
✓ Sin nulos en claves primarias
✓ Sin duplicados en claves primarias

✅ Validaciones finales pasadas


## 9. Conclusiones y Exportación
Generamos un resumen del proceso de limpieza, exportamos los datasets limpios y documentamos métricas clave.

⬆️ Volver al índice

In [9]:
print("💾 Exportación de datasets limpios:\n")

# Exportar datasets limpios
clientes.to_csv(DIR_DATASETS_LIMPIOS / 'clientes_clean.csv', index=False)
productos.to_csv(DIR_DATASETS_LIMPIOS / 'productos_clean.csv', index=False)
ventas.to_csv(DIR_DATASETS_LIMPIOS / 'ventas_clean.csv', index=False)
detalle.to_csv(DIR_DATASETS_LIMPIOS / 'detalle_clean.csv', index=False)

print(f"✓ clientes_clean.csv: {len(clientes):,} filas")
print(f"✓ productos_clean.csv: {len(productos):,} filas")
print(f"✓ ventas_clean.csv: {len(ventas):,} filas")
print(f"✓ detalle_clean.csv: {len(detalle):,} filas")

# Resumen de limpieza
print("\n📊 Resumen del proceso de limpieza:\n")
resumen = pd.DataFrame({
    'Tabla': ['clientes', 'productos', 'ventas', 'detalle_ventas'],
    'Original': [len(orig_clientes), len(orig_productos), len(orig_ventas), len(orig_detalle)],
    'Limpio': [len(clientes), len(productos), len(ventas), len(detalle)],
    'Filas_eliminadas': [
        len(orig_clientes) - len(clientes),
        len(orig_productos) - len(productos),
        len(orig_ventas) - len(ventas),
        len(orig_detalle) - len(detalle)
    ]
})
resumen['%_Retenido'] = (resumen['Limpio'] / resumen['Original'] * 100).round(2)
print(resumen.to_string(index=False))

# Exportar resumen
resumen.to_csv(DIR_EXPORT / 'resumen_limpieza.csv', index=False)
print(f"\n✓ Resumen exportado: {DIR_EXPORT / 'resumen_limpieza.csv'}")

print("\n✅ Proceso de limpieza completo. Datasets listos para EDA y modelado.")

💾 Exportación de datasets limpios:

✓ clientes_clean.csv: 1,187 filas
✓ productos_clean.csv: 139 filas
✓ ventas_clean.csv: 11,728 filas
✓ detalle_clean.csv: 11,533 filas

📊 Resumen del proceso de limpieza:

         Tabla  Original  Limpio  Filas_eliminadas  %_Retenido
      clientes      1296    1187               109       91.59
     productos       151     139                12       92.05
        ventas     12960   11728              1232       90.49
detalle_ventas     12960   11533              1427       88.99

✓ Resumen exportado: c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion\export\resumen_limpieza.csv

✅ Proceso de limpieza completo. Datasets listos para EDA y modelado.
✓ clientes_clean.csv: 1,187 filas
✓ productos_clean.csv: 139 filas
✓ ventas_clean.csv: 11,728 filas
✓ detalle_clean.csv: 11,533 filas

📊 Resumen del proceso de limpieza:

         Tabla  Original  Limpio  Filas_eliminadas  %_Retenido
      clientes      1296    1187        

### Regla adicional: Imputación de valores faltantes en `ciudad`
Esta celda aplica una regla de negocio: cuando la columna `ciudad` esté vacía o nula en `clientes_clean.csv`, se imputará con "Buenos Aires" (in-place). La escritura se realiza sobre `datasets_limpios/clientes_clean.csv` y se deja un listado de casos imputados en `export/`.

In [10]:
from pathlib import Path
import pandas as pd

# Rutas
try:
    base_export = Path(DIR_EXPORT)
except NameError:
    base_export = Path('export')
base_export.mkdir(exist_ok=True)

try:
    path_clientes = Path(DIR_DATASETS_LIMPIOS) / 'clientes_clean.csv'
except NameError:
    path_clientes = Path('datasets_limpios') / 'clientes_clean.csv'

# Cargar desde disco para robustez
_dfc = pd.read_csv(path_clientes)

# Normalizar y aplicar imputación
_mask = _dfc['ciudad'].replace('', pd.NA).isna()
_ids = _dfc.loc[_mask, 'id_cliente'].tolist()
_dfc.loc[_mask, 'ciudad'] = 'Buenos Aires'

# Guardar IN-PLACE
_dfc.to_csv(path_clientes, index=False, encoding='utf-8')

# Actualizar variable en sesión si existe `clientes`
try:
    clientes.update(_dfc.set_index('id_cliente'))
except Exception:
    pass

# Validación y export de casos
_faltantes_post = int(_dfc['ciudad'].replace('', pd.NA).isna().sum())
if _ids:
    (_dfc[_dfc['id_cliente'].isin(_ids)][['id_cliente','nombre_cliente','email','ciudad','fecha_alta']]
        .to_csv(base_export / 'clientes_ciudad_imputada_bsas.csv', index=False, encoding='utf-8'))

print({'imputados': len(_ids), 'faltantes_ciudad_post': _faltantes_post, 'path': str(path_clientes)})
if _ids:
    print('IDs imputados:', _ids)
else:
    print('No había filas con ciudad faltante.')

{'imputados': 6, 'faltantes_ciudad_post': 0, 'path': 'c:\\Users\\LENOVO\\Desktop\\FUNDAMENTOS DE IA GUAYERD\\Diego Vásquez - Proyecto Aurelion\\datasets_limpios\\clientes_clean.csv'}
IDs imputados: [359, 575, 584, 666, 1013, 1067]


### Regla adicional: eliminar productos con nombre vacío
Eliminamos filas de `productos_clean.csv` donde `nombre_producto` esté vacío o nulo. Se registran las filas removidas en `export/productos_nombre_vacio_eliminados.csv` para trazabilidad.

In [11]:
import pandas as pd
from pathlib import Path

DIR_DATASETS_LIMPIOS = Path.cwd() / 'datasets_limpios'
DIR_EXPORT = Path.cwd() / 'export'
DIR_EXPORT.mkdir(exist_ok=True)

path_productos = DIR_DATASETS_LIMPIOS / 'productos_clean.csv'
df_prod = pd.read_csv(path_productos)
antes = len(df_prod)

mask_vacio = df_prod['nombre_producto'].isna() | (df_prod['nombre_producto'].astype(str).str.strip() == '')
eliminados_df = df_prod.loc[mask_vacio].copy()
df_prod = df_prod.loc[~mask_vacio].copy()
despues = len(df_prod)

if not eliminados_df.empty:
    out_removed = DIR_EXPORT / 'productos_nombre_vacio_eliminados.csv'
    eliminados_df.to_csv(out_removed, index=False, encoding='utf-8')
    print(f'Se eliminaron {len(eliminados_df)} filas con nombre vacío. Archivo de respaldo: {out_removed}')
else:
    print('No se encontraron filas con nombre vacío en productos.')

df_prod.to_csv(path_productos, index=False, encoding='utf-8')
print(f'Registro productos limpio guardado: {path_productos}')
print(f'Filas antes: {antes} | después: {despues}')

# Validación final
assert not (df_prod['nombre_producto'].astype(str).str.strip() == '').any(), 'Persisten nombres vacíos tras la limpieza'
print('Validación OK: no quedan nombres vacíos.')

Se eliminaron 1 filas con nombre vacío. Archivo de respaldo: c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion\export\productos_nombre_vacio_eliminados.csv
Registro productos limpio guardado: c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion\datasets_limpios\productos_clean.csv
Filas antes: 139 | después: 138
Validación OK: no quedan nombres vacíos.


### Regla adicional: propagar CSAT, CES y NPS a `ventas_clean`
Asegura que las columnas `csat_1a5`, `ces_1a5`, `nps_0a10` y `nps_segmento` queden en el export limpio (`CSV/Excel`).

In [12]:
from pathlib import Path
import pandas as pd

# Columnas nuevas esperadas
cols_cx = ['csat_1a5','ces_1a5','nps_0a10','nps_segmento']
for c in cols_cx:
    if c not in ventas.columns:
        ventas[c] = pd.NA

ventas['csat_1a5'] = pd.to_numeric(ventas['csat_1a5'], errors='coerce')
ventas['ces_1a5'] = pd.to_numeric(ventas['ces_1a5'], errors='coerce')
ventas['nps_0a10'] = pd.to_numeric(ventas['nps_0a10'], errors='coerce')
ventas['nps_segmento'] = ventas['nps_segmento'].astype('string')

DIR_DATASETS_LIMPIOS.mkdir(parents=True, exist_ok=True)

# Exportar CSV y Excel preservando columnas
csv_out = DIR_DATASETS_LIMPIOS / 'ventas_clean.csv'
xlsx_out = DIR_DATASETS_LIMPIOS / 'ventas_clean.xlsx'
ventas.to_csv(csv_out, index=False)
with pd.ExcelWriter(xlsx_out, engine='xlsxwriter') as writer:
    ventas.to_excel(writer, index=False, sheet_name='ventas')

print('Exportado ventas limpio con CX ->', csv_out.as_posix())
print('Columnas:', list(ventas.columns))

Exportado ventas limpio con CX -> c:/Users/LENOVO/Desktop/FUNDAMENTOS DE IA GUAYERD/Diego Vásquez - Proyecto Aurelion/datasets_limpios/ventas_clean.csv
Columnas: ['id_venta', 'fecha', 'id_cliente', 'medio_pago', 'canal', 'csat_1a5', 'ces_1a5', 'nps_0a10', 'nps_segmento']


In [13]:
# -- Persistencia de métricas CX en ventas_clean --
import pandas as pd
import numpy as np
from pathlib import Path

cols_cx = ["csat_1a5", "ces_1a5", "nps_0a10", "nps_segmento"]

v_path = DIR_DATASETS / "ventas.xlsx"
vo = ventas.copy()

# Alinear tipos de fecha
if "fecha" in vo.columns:
    vo["fecha"] = pd.to_datetime(vo["fecha"], errors="coerce")

if v_path.exists():
    v_raw = pd.read_excel(v_path)
    v_raw.columns = [str(c).strip() for c in v_raw.columns]
    if "fecha" in v_raw.columns:
        v_raw["fecha"] = pd.to_datetime(v_raw["fecha"], errors="coerce")

    # Merge preferente por id_venta, fallback por (id_cliente, fecha)
    if "id_venta" in vo.columns and "id_venta" in v_raw.columns:
        vo = vo.merge(
            v_raw[["id_venta"] + [c for c in cols_cx if c in v_raw.columns]],
            on="id_venta", how="left", suffixes=("", "_src")
        )
    else:
        merge_keys = [k for k in ["id_cliente", "fecha"] if k in vo.columns and k in v_raw.columns]
        if merge_keys:
            vo = vo.merge(
                v_raw[merge_keys + [c for c in cols_cx if c in v_raw.columns]],
                on=merge_keys, how="left", suffixes=("", "_src")
            )

    # Completar valores CX desde sufijos _src si aplica
    for c in cols_cx:
        if c in vo.columns and f"{c}_src" in vo.columns:
            vo[c] = vo[c].fillna(vo[f"{c}_src"])
            vo.drop(columns=[f"{c}_src"], inplace=True)

# Normalizar rangos
for c in ["csat_1a5", "ces_1a5"]:
    if c in vo.columns:
        vo[c] = pd.to_numeric(vo[c], errors="coerce")
        vo.loc[~vo[c].between(1,5), c] = np.nan

if "nps_0a10" in vo.columns:
    vo["nps_0a10"] = pd.to_numeric(vo["nps_0a10"], errors="coerce")
    vo.loc[~vo["nps_0a10"].between(0,10), "nps_0a10"] = np.nan

# Exportar sobrescribiendo ventas_clean
out_csv = DIR_DATASETS_LIMPIOS / "ventas_clean.csv"
out_xlsx = DIR_DATASETS_LIMPIOS / "ventas_clean.xlsx"
vo.to_csv(out_csv, index=False)
with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
    vo.to_excel(writer, index=False, sheet_name="ventas")

# Resumen rápido
cnt_cx = vo[[c for c in cols_cx if c in vo.columns]].notna().sum()
print("Exportado ventas_clean con CX →", out_csv)
print("No-nulos CX:")
print(cnt_cx.to_string())

Exportado ventas_clean con CX → c:\Users\LENOVO\Desktop\FUNDAMENTOS DE IA GUAYERD\Diego Vásquez - Proyecto Aurelion\datasets_limpios\ventas_clean.csv
No-nulos CX:
csat_1a5        10365
ces_1a5          9607
nps_0a10         2179
nps_segmento     2179
